In [1]:
import numpy as np

In [2]:
import torch 
import h5py

class VSMDataset(torch.utils.data.Dataset):
    """Video Summarizer Dataset
        Datasets: TVSum, Summe, VSUMM, CoSum, Visiocity
    """

    def __init__(self, hdfs_path, split=None, 
                 googlenet=False, 
                 resnext=False, 
                 inceptionv3=False,
                 i3d_rgb=False,
                 i3d_flow=False,
                 resnet3d=False
                ):
        """
        Args:
           hdfs_path (string): path of the hdfs processed data
           split (dict): idxs of the train/test split 
        """
        videos_info = h5py.File(hdfs_path)
        self.labels = {}
        self.data = {}
        
        keys_to_avoid = ['gtscore', 'gtsummary', 'user_summary']
        
        if not googlenet:
            keys_to_avoid.append('features')
        if not resnext:
            keys_to_avoid.append('features_rn')
        if not inceptionv3:
            keys_to_avoid.append('features_iv3')
        if not i3d_rgb:
            keys_to_avoid.append('features_rgb')
        if not i3d_flow:
            keys_to_avoid.append('features_flow')
        if not resnet3d:
            keys_to_avoid.append('features_3D')
        
        for it, video in enumerate(list(videos_info)):
            self.labels[it] = dict((key, videos_info[video][key][...])for key in list(videos_info[video]) if key in ('gtscore', 'gtsummary', 'user_summary') )
            self.data[it] = dict((key, videos_info[video][key][...])for key in list(videos_info[video]) if key not in keys_to_avoid )   
            
            if "video_name" in self.data[it].keys():
                self.data[it]["video_name"] = str(self.data[it]["video_name"]) 
            
        if split:
            #TO-DO
            pass
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        X = self.data[index]
        y = self.labels[index]        
        return X, y
    
    def get_feature(self, index, feature):
        X = self.data[index][feature]
        return X

def show_sample(idx, hdfs_path):

    vsm_dataset = VSMDataset(hdfs_path=hdfs_path)
    video_info, label = vsm_dataset[idx]

    return video_info, label



In [3]:
paths = [
        "/home/shuaman/video_sm/video_summarization/src/dataset_ovp_processed.h5",
        "/home/shuaman/video_sm/video_summarization/src/dataset_summe_processed.h5",
        "/home/shuaman/video_sm/video_summarization/src/dataset_tvsum_processed.h5",
        "/home/shuaman/video_sm/video_summarization/src/dataset_youtube_processed.h5",
        "/home/shuaman/video_sm/video_summarization/src/dataset_cosum_processed.h5",
        ]

In [4]:
features_rn = []
features_iv3 = []
features_3D = []
features= []
features_rgb = []
features_flow= []

for feature in ["features_rn", "features_iv3", "features_3D", "features", "features_rgb", "features_flow"]:
    for path in paths:
        training_set = VSMDataset(path, googlenet=True, resnext=True, inceptionv3=True,
                                 i3d_rgb=True, i3d_flow=True, resnet3d=True)

        for i in range(len(training_set)):
            feat = training_set.get_feature(i, feature)
            
            for nframe in range(len(feat)):        
                if feature=="features_rn":
                    features_rn.append(feat[nframe])
                elif feature=="features_iv3":
                    features_iv3.append(feat[nframe])
                elif feature=="features_3D":
                    features_3D.append(feat[nframe])
                elif feature=="features":
                    features.append(feat[nframe])
                elif feature=="features_rgb":
                    features_rgb.append(feat[nframe])
                elif feature=="features_flow":
                    features_flow.append(feat[nframe])
                
features_rn = np.array(features_rn)    
features_iv3 = np.array(features_iv3)    
features_3D = np.array(features_3D)    
features = np.array(features)    
features_rgb = np.array(features_rgb)    
features_flow = np.array(features_flow)    

In [5]:
features.shape

(70676, 1024)

In [6]:
features_rgb.shape

(54342, 1024)

In [7]:
features_flow.shape

(54137, 1024)

In [8]:
features_iv3.shape

(70676, 2048)

In [9]:
features_3D.shape

(54342, 2048)

In [10]:
features_rn.shape

(70676, 2048)

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [13]:
def fit_transform_data(descriptors_orig, n_components):
    """
    descriptors_orig: descriptores directamente obtenidos del modelo
    n_components: numero de componentes de pca
    """
    normalizer = StandardScaler()
    normalizer = normalizer.fit(descriptors_orig)
    descriptors = normalizer.transform(descriptors_orig)
    pca_transformer = PCA(n_components = n_components, random_state=0)
    pca_transformer = pca_transformer.fit(descriptors)
    descriptors = pca_transformer.transform(descriptors)
    return descriptors, normalizer, pca_transformer

In [14]:
features_iv3_pca, normalizer_iv3, pca_iv3 = fit_transform_data(descriptors_orig=features_iv3, n_components=1024)
features_3D_pca, normalizer_3D, pca_3D = fit_transform_data(descriptors_orig=features_3D, n_components=1024)
features_rn_pca, normalizer_rn, pca_rn = fit_transform_data(descriptors_orig=features_rn, n_components=1024)

_, normalizer_googlenet, _ = fit_transform_data(descriptors_orig=features, n_components=1024)
_, normalizer_rgb, _ = fit_transform_data(descriptors_orig=features_rgb, n_components=1024)
_, normalizer_flow, _ = fit_transform_data(descriptors_orig=features_flow, n_components=1024)

In [15]:
features_iv3_pca.shape

(70676, 1024)

In [16]:
features_3D_pca.shape

(54342, 1024)

In [17]:
features_rn_pca.shape

(70676, 1024)

In [18]:
features_iv3_pca[-1]

array([ 1.9938809 , -1.169457  , -4.483862  , ..., -0.12678224,
       -0.59480363, -0.22077972], dtype=float32)

In [19]:
pca_iv3.transform(normalizer_iv3.transform(features_iv3))[-1]

array([ 1.9938809 , -1.169457  , -4.483862  , ..., -0.12678224,
       -0.59480363, -0.22077972], dtype=float32)

In [20]:
import pickle

In [21]:
transformations = {
    "normalizer_iv3": normalizer_iv3,
    "pca_iv3": pca_iv3,
    "normalizer_3D": normalizer_3D,
    "pca_3D": pca_3D,
    "normalizer_rn": normalizer_rn,
    "pca_rn": pca_rn,
    "normalizer_googlenet": normalizer_googlenet,
    "normalizer_rgb": normalizer_rgb,
    "normalizer_flow": normalizer_flow
}

In [23]:
filename = "transformations.pk"

pickle.dump(transformations, open(filename, 'wb'))